In [13]:
import json
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

In [14]:
# load dataset
filename = 'text_and_label/json_vectorized.json'

X = []
y = []
with open(filename) as json_file:
    data = json.load(json_file)
    
    for key, value in data.items():
        X.append(value["vectorized"])
        y.append(value["label"])
    
print(X[0])
print(y[0])

[12, 0, 0, 9, 0, 6, 15, 0, 0, 0, 8, 18, 20, 4, 0, 0, 0, 19, 0, 0, 17, 0, 1, 24, 7, 0, 23, 16, 0, 13, 14, 21, 22, 5]
2


In [23]:
top_words = 8000
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [24]:
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [34]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 500, 32)           256000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 309,301
Trainable params: 309,301
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
model.fit(X_train, y_train, epochs=6, batch_size=16)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/6


InvalidArgumentError: Received a label value of 4 which is outside the valid range of [0, 1).  Label values: 3 3 2 3 3 2 3 3 4 3 4 2 4 3 4 4
	 [[Node: loss_4/dense_5_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits = SparseSoftmaxCrossEntropyWithLogits[T=DT_FLOAT, Tlabels=DT_INT64, _class=["loc:@train...s_grad/mul"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](loss_4/dense_5_loss/Log, loss_4/dense_5_loss/Cast)]]